In [ ]:
# data_ingest_and_preparation.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml import Pipeline

# ==================== CONFIGURATION ====================
DELTA_TABLE_NAME = "house_price_delta"
OUTPUT_DELTA_TABLE = "house_price_scaled_delta"

# Global variables
global TRAINING_DATA_DF
global FEATURE_COLUMNS
TRAINING_DATA_DF = None
FEATURE_COLUMNS = None

# ==================== FUNCTIONS ====================

def initialize_spark():
    """SparkSession को इनिशियलाइज़ करता है"""
    if 'spark' in globals() and isinstance(globals()['spark'], SparkSession):
        return globals()['spark']
    try:
        spark = SparkSession.builder.appName("DataIngestAndPreparation").getOrCreate()
        return spark
    except Exception as e:
        print(f"SparkSession initialization failed: {e}")
        return None

def ingest_data(spark: SparkSession, table_name: str):
    """डेल्टा टेबल से हाउस प्राइस डेटा को इंगेस्ट करता है"""
    if spark is None:
        print("❌ SparkSession उपलब्ध नहीं है। डेटा इंगेस्ट नहीं किया जा सकता।")
        return None, None
        
    print(f"डेल्टा टेबल '{table_name}' से डेटा लोड हो रहा है...")

    try:
        house_df = spark.read.format("delta").table(table_name)
        
        feature_cols = ['sq_feet', 'num_bedrooms', 'num_bathrooms', 'year_built', 'location_score']
        label_col = 'price'
        
        ingested_df = house_df.select(*feature_cols, col(label_col).alias("label"))
        
        print(f"✅ डेटा सफलतापूर्वक इंगेस्ट हुआ। कुल पंक्तियाँ: {ingested_df.count()}")
        ingested_df.printSchema()
        
        return ingested_df, feature_cols
        
    except Exception as e:
        print(f"डेटा इंगेस्ट में त्रुटि: {e}")
        return None, None


def prepare_data(spark, df, feature_cols):
    """फीचर स्केलिंग (StandardScaler) लागू करता है"""
    print("\n⚙️ फीचर स्केलिंग शुरू हो रही है...")

    # Step 1: Assemble features into a single vector
    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_vector")

    # Step 2: Apply StandardScaler
    scaler = StandardScaler(inputCol="features_vector", outputCol="scaled_features", withMean=True, withStd=True)

    # Step 3: Pipeline for transformation
    pipeline = Pipeline(stages=[assembler, scaler])
    model = pipeline.fit(df)
    scaled_data = model.transform(df)

    # Step 4: Scaled features को अलग-अलग कॉलम में बदलना
    from pyspark.ml.functions import vector_to_array
    scaled_data = scaled_data.withColumn("scaled_array", vector_to_array(col("scaled_features")))

    for i, c in enumerate(feature_cols):
        scaled_data = scaled_data.withColumn(f"{c}_scaled", col("scaled_array")[i])

    # Step 5: Final scaled DataFrame बनाना (scaled features + label)
    final_cols = [f"{c}_scaled" for c in feature_cols] + ["label"]
    scaled_final_df = scaled_data.select(*final_cols)

    print("✅ फीचर स्केलिंग पूरी हुई।")
    scaled_final_df.show(5)

    # Step 6: Scaled data को Delta table में सेव करना
    scaled_final_df.write.format("delta").mode("overwrite").saveAsTable(OUTPUT_DELTA_TABLE)
    print(f"✅ स्केल्ड डेटा Delta table '{OUTPUT_DELTA_TABLE}' में सेव हो गया।")

    return scaled_final_df


# ==================== EXECUTION ====================

if __name__ == "__main__":
    spark_session = initialize_spark()
    ingested_data_df, features = ingest_data(spark_session, DELTA_TABLE_NAME)

    if ingested_data_df is not None:
        globals()['TRAINING_DATA_DF'] = ingested_data_df
        globals()['FEATURE_COLUMNS'] = features
        print("✅ डेटा को ग्लोबल वेरिएबल्स में सेव किया गया।")

        # Feature scaling step
        scaled_df = prepare_data(spark_session, ingested_data_df, features)

    else:
        print("❌ डेटा इंगेस्ट विफल रहा।")
